# CCC

In [1]:
import time
import numpy as np

start_time = time.time()
parameters = np.array([[0.05, 0.15, 0.75],
                      [0.1, 0.25, 0.6],
                      [0.15, 0.15, 0.8]])

correlation_matrix =  np.array([[[1, 0.6, 0.4],
                                 [0.6, 1, -0.3],
                                 [0.4, -0.3, 1]]])


decomp = np.linalg.cholesky(correlation_matrix)
decomp
# Number of observations
T = 1000000
K = correlation_matrix.shape[1]
# Initialize arrays to store the processes and variances
processes = np.zeros((K, T))
variances = np.zeros((K, T))
innovations = np.zeros((K, T))

# Initial variance (can be set equal to the long-term variance)
variances[:, 0] = parameters[:, 0] / (1 - parameters[:, 1] - parameters[:, 2])

# Simulate the GARCH processes
for t in range(1, T):
    # Simulate standard normal innovations
    z = np.random.normal(0, 1, K)
    # Apply the Cholesky decomposition to introduce correlation
    correlated_z = decomp @ z
    # Update the variance
    variances[:, t] = parameters[:, 0] + parameters[:, 1] * (processes[:, t-1]**2) + parameters[:, 2] * variances[:, t-1]
    # Calculate the process values
    processes[:, t] = np.sqrt(variances[:, t]) * correlated_z[:]

# Output the first few values to check
print(processes[:, :5])

# End timing
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

# Print elapsed time with 5 digits after the decimal point
print(f"Elapsed time: {elapsed_time:.5f} seconds")


[[ 0.          0.43349681 -0.22774791  0.25227031  0.41850353]
 [ 0.          0.36503723 -0.03801007 -0.09632761 -0.24085654]
 [ 0.         -0.55668597 -1.86914962  0.1329955   1.45124257]]
Elapsed time: 9.74573 seconds


In [2]:
def standard_deviations(data, univariate_parameters):
    K,T = data.shape
    sigmas = np.zeros((K,T))
    sigmas[:,0] = np.var(data, axis=1)
    for t in range(1, T):
        sigmas[:,t] = univariate_parameters[:,0] + univariate_parameters[:, 1] * data[:, t-1]**2 + univariate_parameters[:, 2] * sigmas[:, t-1]    
    return np.sqrt(sigmas)

def calculate_res(data, sigma):
    return data / sigma

def density(residuals):
    K, T = residuals.shape
    correlation_matrix = np.sum(np.einsum('it,jt->ijt', residuals, residuals), axis=-1) / T
    return correlation_matrix
    
def cholesky_scale(matrix):
    K, E = matrix.shape
    P = np.linalg.cholesky(matrix)
    for j in range(K):
        sum = np.sum(P[j, :j] ** 2)
        P[j,j] = np.sqrt(1-sum) if 1 - sum > 0 else 0
    scaled = np.dot(P, P.T)
    return scaled




start_time = time.time()


std = standard_deviations(processes, parameters)
res = calculate_res(processes, std)
correlation_matrix = density(res)
scaled = cholesky_scale(correlation_matrix)
print(scaled)
# End timing
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

# Print elapsed time with 5 digits after the decimal point
print(f"Elapsed time: {elapsed_time:.5f} seconds")


[[ 1.          0.60002472  0.40001577]
 [ 0.60002472  1.         -0.2997962 ]
 [ 0.40001577 -0.2997962   1.        ]]
Elapsed time: 3.61978 seconds


# Multi state


In [5]:
import time
import numpy as np

start_time = time.time()
parameters = np.array([[0.05, 0.15, 0.75],
                      [0.1, 0.25, 0.6],
                      [0.15, 0.15, 0.8],
                      [0.2, 0.3, 0.65]])

true_correlation_matrix =  matrices = np.array([
    # High correlations
    [[1, 0.8, 0.7, 0.6],
     [0.8, 1, 0.65, 0.55],
     [0.7, 0.65, 1, 0.5],
     [0.6, 0.55, 0.5, 1]],
    
    # Low correlations
    [[1, 0.1, 0.15, 0.05],
     [0.1, 1, 0.2, 0.1],
     [0.15, 0.2, 1, 0.12],
     [0.05, 0.1, 0.12, 1]],
    
    # Negative correlations
    [[1, -0.5, -0.4, -0.3],
     [-0.5, 1, -0.2, -0.1],
     [-0.4, -0.2, 1, -0.25],
     [-0.3, -0.1, -0.25, 1]],
    
    # In between
    [[1, 0.4, -0.3, 0.2],
     [0.4, 1, 0.25, -0.2],
     [-0.3, 0.25, 1, 0.15],
     [0.2, -0.2, 0.15, 1]]
])
diagonal = 0.99
N = 4
K = 4
transition_matrix = diagonal * np.eye(N) + (1-diagonal) * (np.ones((N,N)) - np.eye(N,N)) / (N - 1)
decomp = np.zeros((N, K, K))
for n in range(N):
    decomp[n,:,:] = choleski_form(true_correlation_matrix[n,:,:])


# Number of observations
T = 10000000

# Initialize arrays to store the processes and variances
processes = np.zeros((K, T))
variances = np.zeros((K, T))
states = np.zeros((T))
innovations = np.zeros((K, T))

# Initial variance (can be set equal to the long-term variance)
variances[:, 0] = parameters[:, 0] / (1 - parameters[:, 1] - parameters[:, 2])
states[0] = 0
# Simulate the GARCH processes
for t in range(1, T):
    state = int(states[t-1])
    current_state = np.random.choice(a=[0,1,2,3], p=transition_matrix[state])
    states[t] = current_state
    
    # Simulate standard normal innovations
    z = np.random.normal(0, 1, K)
    # Apply the Cholesky decomposition to introduce correlation
    correlated_z = decomp[current_state,:,:] @ z
    # Update the variance
    variances[:, t] = parameters[:, 0] + parameters[:, 1] * (processes[:, t-1]**2) + parameters[:, 2] * variances[:, t-1]
    # Calculate the process values
    processes[:, t] = np.sqrt(variances[:, t]) * correlated_z[:]











# End timing
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

# Print elapsed time with 5 digits after the decimal point
print(f"Elapsed time: {elapsed_time:.5f} seconds")


Elapsed time: 384.48471 seconds


In [6]:
def choleski_form(matrix):
    return np.linalg.cholesky(matrix)
def standard_deviations(data, univariate_parameters):
    K,T = data.shape
    sigmas = np.zeros((K,T))
    sigmas[:,0] = np.var(data, axis=1)
    for t in range(1, T):
        sigmas[:,t] = univariate_parameters[:,0] + univariate_parameters[:, 1] * data[:, t-1]**2 + univariate_parameters[:, 2] * sigmas[:, t-1]    
    return np.sqrt(sigmas)

def calculate_res(data, sigma):
    return data / sigma

def density(residuals,states):
    K, T = residuals.shape
    sum_states = np.sum(states, axis=-1)
    correlation_matrix = np.sum(np.einsum('it,jt,nt->nijt', residuals, residuals, states), axis=-1) /sum_states
    return correlation_matrix
    
def cholesky_scale(matrix):
    K, E = matrix.shape
    P = np.linalg.cholesky(matrix)
    for j in range(K):
        sum = np.sum(P[j, :j] ** 2)
        P[j,j] = np.sqrt(1-sum) if 1 - sum > 0 else 0
    scaled = np.dot(P, P.T)
    return scaled

def form_states(states, N, T):
    # Ensure states are integers within the correct range
    states = np.clip(states.astype(int), 0, N-1)
    
    # Initialize the state array with zeros
    state_array = np.zeros((N, T))
    
    # Mark ones for active states
    for t, state in enumerate(states):
        state_array[state, t] = 1
    
    return state_array


start_time = time.time()


std = standard_deviations(processes, parameters)
res = calculate_res(processes, std)
true_states = form_states(states, N, T)
correlation_matrix = density(res, true_states)
scaled = np.zeros((N, K, K))
for n in range(N):
    scaled[n,:,:] = cholesky_scale(correlation_matrix[n,:,:])


print(scaled)
# End timing
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

# Print elapsed time with 5 digits after the decimal point
print(f"Elapsed time: {elapsed_time:.5f} seconds")


[[[ 1.          0.8019691   0.70110199  0.60104857]
  [ 0.8019691   1.          0.64973254  0.54983106]
  [ 0.70110199  0.64973254  1.          0.50393924]
  [ 0.60104857  0.54983106  0.50393924  1.        ]]

 [[ 1.          0.10025644  0.15013322  0.04949751]
  [ 0.10025644  1.          0.20025296  0.09852301]
  [ 0.15013322  0.20025296  1.          0.12017861]
  [ 0.04949751  0.09852301  0.12017861  1.        ]]

 [[ 1.         -0.49841708 -0.39797933 -0.29952701]
  [-0.49841708  1.         -0.20194527 -0.09944296]
  [-0.39797933 -0.20194527  1.         -0.24991191]
  [-0.29952701 -0.09944296 -0.24991191  1.        ]]

 [[ 1.          0.40045791 -0.29903808  0.19978641]
  [ 0.40045791  1.          0.2499614  -0.19981725]
  [-0.29903808  0.2499614   1.          0.15010042]
  [ 0.19978641 -0.19981725  0.15010042  1.        ]]]
Elapsed time: 41.94825 seconds


In [ ]:
correlation_matrix

In [ ]:
scaled

In [ ]:
true_correlation_matrix

In [7]:
emission  = scaled - true_correlation_matrix
np.max(emission),np.min(emission), emission

(0.0039392418367781845,
 -0.0019452746091170592,
 array([[[ 0.00000000e+00,  1.96909865e-03,  1.10199299e-03,
           1.04857276e-03],
         [ 1.96909865e-03,  0.00000000e+00, -2.67456206e-04,
          -1.68936372e-04],
         [ 1.10199299e-03, -2.67456206e-04,  0.00000000e+00,
           3.93924184e-03],
         [ 1.04857276e-03, -1.68936372e-04,  3.93924184e-03,
           2.22044605e-16]],
 
        [[ 0.00000000e+00,  2.56437986e-04,  1.33222412e-04,
          -5.02488696e-04],
         [ 2.56437986e-04,  0.00000000e+00,  2.52957837e-04,
          -1.47698584e-03],
         [ 1.33222412e-04,  2.52957837e-04,  0.00000000e+00,
           1.78610192e-04],
         [-5.02488696e-04, -1.47698584e-03,  1.78610192e-04,
          -1.11022302e-16]],
 
        [[ 0.00000000e+00,  1.58292403e-03,  2.02067146e-03,
           4.72987738e-04],
         [ 1.58292403e-03,  0.00000000e+00, -1.94527461e-03,
           5.57036258e-04],
         [ 2.02067146e-03, -1.94527461e-03,  0.00000000

In [ ]:
transition_matrix